In [ ]:
!pip install sentence_transformers
!pip install konlpy

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """
드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
"""

In [ ]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

In [ ]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
print(doc_embedding.shape)
print(candidate_embeddings.shape)

In [ ]:
print(doc)
print(candidates)

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
print(distances)
# keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
# print(keywords)

In [ ]:
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, top_n=5, nr_candidates=10)

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, top_n=5, nr_candidates=30)

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.8)

In [ ]:
docs = """
건설업계의 메타버스(3차원 가상현실) 사랑이 계속되고 있다. 메타버스를 통해 견본주택을 마련하고 채용설명회를 진행하는 등 포스트 코로나 시대를 맞아 온·오프라인 전환에 나서는 것이다.
23일 업계에 따르면 대우건설이 게임엔진을 활용한 가상체험 견본주택인 '메타갤러리' 서비스를 개시한다. 게임엔진(Game Engine)은 게임을 구동시키는데 필요한 다양한 핵심 기능들을 담은 소프트웨어나 소프트웨어 구성요소를 말한다.
이번에 대우건설이 개발한 메타갤러리는 기존 '메타버스 모델하우스'나 '사이버 모델하우스'와는 차원이 다른 서비스를 제공한다. 가장 큰 차이점은 1인칭 시점으로 게임을 하는 것처럼 사용자가 조작을 통해 가상공간을 직접 돌아다니며 현실감 있게 상품을 볼 수 있다는 점이다.
일반적인 사이버 모델하우스는 건설사가 지정해 놓은 각 실(거실, 욕실, 방 등)의 특정 지점에서 상품을 둘러 볼 수 있다. 설정된 지점에서 360도 뷰(View)로 실을 둘러 볼 수 있지만, 직접 눈으로 보는 것과는 다르게 화면 왜곡이 발생하고 지정된 장소 외에는 이동할 수 없다는 단점이 있다.
대우건설은 기존 서비스들의 문제점을 개선하고 사용자가 모델하우스를 직접 가서 보는 것과 같은 체험을 할 수 있도록 게임개발용 유니티 엔진(Unity Engine)과 3차원 BIM(Building Information Modeling)모델을 활용했다.
롯데건설은 업계 최초로 메타버스 플랫폼 게더 타운(Gather Town)을 활용해 채용설명회를 진행해 화재를 모았다. 여기에 더 나아가 신입사원 입문 교육까지 메타버스로 진행했다. 신입사원들은 아바타로 음성 대화와 화상연결, 화면공유 등을 통해 자유롭게 동기들과 소통하며 교육 프로그램에 참여했다.
롯데건설은 올해 1월 신입사원 채용설명회에서도 메타버스 플랫폼을 활용했다. 롯데건설 시그니처 건물인 롯데월드타워와 롯데캐슬을 배경으로 한 가상세계를 구현해 직무 상담과 라이브 토크쇼를 화상 대화로 진행했다.
프롭테크 기업 직방은 가상 오피스에 기업들을 입주시키고 있다. 직방은 최근 식품기업 아워홈과 원격근무 활성화를 위한 업무협약(MOU)을 맺었다. 직방은 오프라인 근무 환경을 대체한 자체 개발 메타버스 공간 ‘메타폴리스’ 일부를 아워홈에 임대 제공하는 것이다.
이미 직방은 지난해 본사와 사무실을 완전히 없애고 전 직원이 메타폴리스에서 원격 근무를 하며 가상공간이 충분히 오프라인 근무 환경을 대체할 수 있다고 증명한 바 있다. 직방은 메타버스, 3D, VR 등 자체 보유한 IT 기술을 활용해 아워홈이 원격근무 환경에서 업무 효율성을 극대화할 수 있도록 지원한다.
박지혜 산업연구원 연구원은 "시장 초기 단계에서 게임, 가상공연 등 콘텐츠 산업을 중심으로 메타버스 플랫폼이 주를 이루고 있지만, 향후 다양한 산업으로 확대될 것"이라며 "제조 등 다양한 산업수요에 선제적 대응해 산업별 생산성 제고를 위한 메타버스 활용방안 마련이 필요하다"고 말했다.
"""

In [ ]:
tokenized_doc = okt.pos(docs)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

In [ ]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

In [ ]:
doc_embedding = model.encode([docs])
candidate_embeddings = model.encode(candidates)

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, top_n=5, nr_candidates=30)

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.8)